In [ ]:
#Import
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import threading
import time
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timezone
from datetime import timedelta


#Class for Interactive Brokers Connection
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
    #Listen for realtime bars
    def updateMktDepth(self, reqId, position: int, operation: int, side: int, price: float, size):
        bot.market_depth_update(reqId, position, operation, side, price, size)
        

#Bot Logic
class Bot():
    ib = None
    
    def __init__(self, Minute):
        self.Minute = Minute
        self.insert_list = []
        self.ask_volume = []
        self.bid_volume = []
        self.ask_price = []
        self.bid_price = []
        
        #Connect to IB on init
        self.ib = IBapi()
        self.ib.connect ('127.0.0.1', 7496, 1)
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        ib_thread.start()
        time.sleep(1)
        
        #Create our IB contract object
        contractEU = Contract()
        contractEU.symbol = "EUR"
        contractEU.secType ="CASH"
        contractEU.exchange = "IDEALPRO"
        contractEU.currency = "USD"

        # Request Market Data
        self.ib.reqMktDepth(0, contractEU, 20, False, [])
    
    #Listen to socket in seperate thread
    def run_loop(self):
        self.ib.run()

    #Pass realtime bar data back to our bot object
    def market_depth_update(self, reqId, position, operation, side, price, size):
        #operation type
        if operation==0 :
            operation='insert'
        elif operation==1 :
            operation='update'
        else:
            operation='delete'
        
        #oder type
        if side==0:
            side='ask'
        else:
            side='bid'

        DateTime = datetime.now(timezone(timedelta(hours=+2)))
        minute = int(DateTime.strftime("%M"))
        
        
        if minute != self.Minute:
            ask_volume_T=[]
            for volumes in self.ask_volume:
                sommes=0
                nb=0
                for k in range(len(volumes)):
                    sommes+=volumes[k]
                    nb=k
                if nb !=0:
                    ask_volume_T.append(sommes/nb)
                else:
                    ask_volume_T.append(sommes)
            
            
            bid_volume_T=[]
            for volumes in self.bid_volume:
                sommes=0
                nb=0
                for k in range(len(volumes)):
                    sommes+=volumes[k]
                    nb=k
                if nb !=0:
                    bid_volume_T.append(sommes/nb)
                else:
                    bid_volume_T.append(sommes)
            
            print("minute =", minute)
            print("Minute", self.Minute)
            plt.figure()
            plt.style.use('ggplot')
            plt.bar(self.ask_price, ask_volume_T, alpha=0.5, label='ask', color='green', width=0.0001)
            plt.bar(self.bid_price, bid_volume_T, alpha=0.5, label='bid', color='red', width=0.0001)
            plt.legend(loc='upper right')
            plt.xlabel('price')
            plt.ylabel('volume')
            plt.title('average volume for price of bid orders')
            plt.show()

            
            self.ask_volume = []
            self.ask_price = []
            self.bid_volume = []
            self.bid_price = []
            self.Minute = minute

        print(DateTime.strftime("%m-%d-%Y, %H:%M:%S.%f"))
        if side=='ask' and size>=10000:
            if float(price) in self.ask_price:
                self.ask_volume[self.ask_price.index(price)].append(size)
            else:
                self.ask_volume.append([size])
                self.ask_price.append(price)
        elif size>=10000:
            if float(price) in self.bid_price:
                self.bid_volume[self.bid_price.index(price)].append(size)
            else:
                self.bid_volume.append([size])
                self.bid_price.append(price)
        
            
            
#Start bot

hour = int(datetime.now(timezone(timedelta(hours=+2))).strftime('%H'))
Minute = int(datetime.now(timezone(timedelta(hours=+2))).strftime('%M'))
day = datetime.now(timezone(timedelta(hours=+2))).strftime('%a')

if True:#day != 'Sat' and day != 'Sun' and day != 'Fri':
    bot = Bot(Minute)
elif day=='Fri' and hour<22:
    bot = Bot(Minute)
elif day=='Fri' and hour==22 and Minute<55:
    bot = Bot(Minute)